In [ ]:
import tkinter as tk
import random
from PIL import Image, ImageTk
import os
import numpy as np

# TODO 1: Implement base game (splitting, insurance)
# TODO 2: Implement AI
# TODO 3: Implement Napoleon side bets (crazy 7, bust it, 21+3, magic match)
# TODO 4: Reimplement AI with Napoleon side bets
# TODO 5: Implement card shuffling like Napoleon and card counting
# TODO 6: Implement card counting AI


In [8]:
CARD_VALUES = {
    '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10,
    'J': 10, 'Q': 10, 'K': 10, 'A': 11
}

# Load card images
def load_card_images():
    images = {}
    suits = ['hearts', 'diamonds', 'clubs', 'spades']
    ranks = list(CARD_VALUES.keys())
    for suit in suits:
        for rank in ranks:
            filename = f'cards/{rank}_of_{suit}.png'
            if os.path.exists(filename):
                image = Image.open(filename)
                image = image.resize((72, 96), Image.LANCZOS)
                images[f'{rank}_of_{suit}'] = ImageTk.PhotoImage(image)
    # Load back of card image
    back_image = Image.open('cards/back.png')
    back_image = back_image.resize((72, 96), Image.LANCZOS)
    images['back'] = ImageTk.PhotoImage(back_image)
    return images

# TODO: Handle card shuffling like Napoleon
# Create deck of cards
def create_deck():
    suits = ['hearts', 'diamonds', 'clubs', 'spades']
    ranks = list(CARD_VALUES.keys())
    deck = []
    for suit in suits:
        for rank in ranks:
            deck.append(f'{rank}_of_{suit}')
    random.shuffle(deck)
    return deck

# Calculate hand value
def calculate_hand(hand):
    min_value = 0
    aces = 0
    for card in hand:
        rank = card.split('_of_')[0]
        if rank == 'A':
            aces += 1
            min_value += 1  # Count ace as 1
        else:
            min_value += CARD_VALUES.get(rank, 0)
    max_value = min_value
    for _ in range(aces):
        if max_value + 10 <= 21:
            max_value += 10
    return min_value, max_value

In [9]:
class BlackjackGame:
    def __init__(self, master):
        self.master = master
        self.master.title("Blackjack")
        self.master.geometry("400x575")  # Set fixed window size
        self.master.resizable(False, False)  # Disable window resizing
        self.balance = 10
        self.card_images = load_card_images()
        self.create_widgets()
        self.deck = []
        self.player_hand = []
        self.dealer_hand = []
        self.bet = 0

    def create_widgets(self):
        self.playing_field_color = "#228B22"  # Green color for playing field and toolbar

        self.master.config(bg=self.playing_field_color)

        self.balance_label = tk.Label(self.master, text=f"Balance: {self.balance} euros", bg=self.playing_field_color, font=("Helvetica", 10, "bold"))
        self.balance_label.pack(pady=10)

        self.bet_label = tk.Label(self.master, text="Place your bet (1-5000 euros):", bg=self.playing_field_color, font=("Helvetica", 10, "bold"))
        self.bet_label.pack(pady=10)

        self.bet_entry = tk.Entry(self.master)
        self.bet_entry.pack(pady=10)

        self.bet_button = tk.Button(self.master, text="Place Bet", command=self.place_bet)
        self.bet_button.pack(pady=10)

        self.dealer_frame = tk.Frame(self.master, bg=self.playing_field_color)
        self.dealer_frame.pack(pady=10)
        self.dealer_label = tk.Label(self.dealer_frame, text="Dealer's Hand:", anchor='center', bg=self.playing_field_color, font=("Helvetica", 10, "bold"))
        self.dealer_label.pack()
        self.dealer_cards = tk.Frame(self.dealer_frame, bg=self.playing_field_color)
        self.dealer_cards.pack()

        self.player_frame = tk.Frame(self.master, bg=self.playing_field_color)
        self.player_frame.pack(pady=10)
        self.player_label = tk.Label(self.player_frame, text="Your Hand:", anchor='center', bg=self.playing_field_color, font=("Helvetica", 10, "bold"))
        self.player_label.pack()
        self.player_cards = tk.Frame(self.player_frame, bg=self.playing_field_color)
        self.player_cards.pack()

        # Create button frame to hold action buttons
        self.button_frame = tk.Frame(self.master, bg=self.playing_field_color)
        
        self.double_button = tk.Button(self.button_frame, text="Double", command=self.player_double)
        self.double_button.pack(side=tk.LEFT, padx=5)

        self.hit_button = tk.Button(self.button_frame, text="Hit", command=self.player_hit)
        self.hit_button.pack(side=tk.LEFT, padx=5)

        self.stand_button = tk.Button(self.button_frame, text="Stand", command=self.player_stand)
        self.stand_button.pack(side=tk.LEFT, padx=5)

        self.message_label = tk.Label(self.master, text="", bg=self.playing_field_color, font=("Helvetica", 10, "bold"))
        self.message_label.pack(pady=10)

        self.button_frame.pack_forget()

    def start_game(self):
        self.message_label.config(text="")
        self.deck = create_deck()
        self.player_hand = []
        self.dealer_hand = []

        self.player_hand.append(self.deck.pop())
        self.dealer_hand.append(self.deck.pop())
        self.player_hand.append(self.deck.pop())
        self.dealer_hand.append(self.deck.pop())

        self.update_hand_labels()

        player_min, player_max = calculate_hand(self.player_hand)
        if player_max == 21:
            winnings = self.bet * 2.5
            self.message_label.config(text=f"Blackjack! You win {winnings} euros.")
            self.balance += winnings
            self.balance_label.config(text=f"Balance: {self.balance} euros")
            self.show_final_hands()
            self.end_round()
        else:
            self.button_frame.pack(pady=10)

    def place_bet(self):
        try:
            if not isinstance(self.bet_entry.get(), int):
                self.message_label.config(text="Please enter a valid bet amount.")
            self.bet = int(self.bet_entry.get())
            if self.bet <= 0 or self.bet > 5000 or self.bet > self.balance:
                self.message_label.config(text="Invalid bet amount.")
            else:
                self.balance -= self.bet
                self.balance_label.config(text=f"Balance: {self.balance} euros")
                self.bet_entry.delete(0, tk.END)
                self.start_game()
        except ValueError:
            self.message_label.config(text="Please enter a valid bet amount.")

    def update_hand_labels(self):
        # Update player hand
        for widget in self.player_cards.winfo_children():
            widget.destroy()
        min_value, max_value = calculate_hand(self.player_hand)
        if min_value == max_value:
            self.player_label.config(text=f"Your Hand: {min_value}")
        else:
            self.player_label.config(text=f"Your Hand: {min_value}-{max_value}")
        for card in self.player_hand:
            img = self.card_images.get(card, self.card_images['back'])
            tk.Label(self.player_cards, image=img, bg=self.playing_field_color).pack(side=tk.LEFT)

        # Update dealer hand
        for widget in self.dealer_cards.winfo_children():
            widget.destroy()
        # Show only first dealer card
        first_card = self.dealer_hand[0]
        img = self.card_images.get(first_card, self.card_images['back'])
        tk.Label(self.dealer_cards, image=img, bg=self.playing_field_color).pack(side=tk.LEFT)
        # Show back image for hidden card
        tk.Label(self.dealer_cards, image=self.card_images['back'], bg=self.playing_field_color).pack(side=tk.LEFT)
        # Calculate the value of the revealed card only
        rank = first_card.split('_of_')[0]
        if rank == 'A':
            min_value = 1
            max_value = 11
        else:
            min_value = max_value = CARD_VALUES.get(rank, 0)
        if min_value == max_value:
            self.dealer_label.config(text=f"Dealer's Hand: {min_value}")
        else:
            self.dealer_label.config(text=f"Dealer's Hand: {min_value}-{max_value}")

    def player_hit(self):
        self.player_hand.append(self.deck.pop())
        self.update_hand_labels()
        min_value, max_value = calculate_hand(self.player_hand)
        if min_value == max_value == 21:
            self.button_frame.pack_forget()
            self.dealer_turn()
        if min_value > 21 and max_value > 21:
            self.message_label.config(text=f"You bust and lose {self.bet} euros. Dealer wins.")
            self.button_frame.pack_forget()
            self.end_round()

    def player_stand(self):
        self.button_frame.pack_forget()
        self.dealer_turn()

    def player_double(self):
        if self.balance >= self.bet:
            # Double the bet
            self.balance -= self.bet
            self.bet *= 2
            self.balance_label.config(text=f"Balance: {self.balance} euros")
            # Deal one more card to the player
            self.player_hand.append(self.deck.pop())
            self.update_hand_labels()
            self.button_frame.pack_forget()
            min_value, max_value = calculate_hand(self.player_hand)
            if min_value > 21:
                self.message_label.config(text=f"You bust and lose {self.bet} euros. Dealer wins.")
                self.end_round()
            else:
                self.dealer_turn()
        else:
            self.message_label.config(text="Not enough balance to double the bet.")
            
    def dealer_turn(self):
        dealer_min_value, dealer_max_value = calculate_hand(self.dealer_hand)
        dealer_value = dealer_max_value if dealer_max_value <= 21 else dealer_min_value
        while dealer_value < 17:
            self.dealer_hand.append(self.deck.pop())
            dealer_min_value, dealer_max_value = calculate_hand(self.dealer_hand)
            dealer_value = dealer_max_value if dealer_max_value <= 21 else dealer_min_value
        self.show_final_hands()
        self.determine_winner()

    def show_final_hands(self):
        # Show all dealer cards
        for widget in self.dealer_cards.winfo_children():
            widget.destroy()
        for card in self.dealer_hand:
            img = self.card_images.get(card, self.card_images['back'])
            tk.Label(self.dealer_cards, image=img, bg=self.playing_field_color).pack(side=tk.LEFT)
        # Calculate best dealer hand value
        min_value, max_value = calculate_hand(self.dealer_hand)
        dealer_value = max_value if max_value <= 21 else min_value
        self.dealer_label.config(text=f"Dealer's Hand: {dealer_value}")

        # Update player hand label with best value
        min_value, max_value = calculate_hand(self.player_hand)
        player_value = max_value if max_value <= 21 else min_value
        self.player_label.config(text=f"Your Hand: {player_value}")

    def determine_winner(self):
        # Get best possible values for player and dealer
        player_min, player_max = calculate_hand(self.player_hand)
        dealer_min, dealer_max = calculate_hand(self.dealer_hand)
        player_value = player_max if player_max <= 21 else player_min
        dealer_value = dealer_max if dealer_max <= 21 else dealer_min

        # Determine outcomes
        if player_value > 21:
            self.message_label.config(text=f"You bust and lose {self.bet} euros. Dealer wins.")
        elif dealer_value > 21:
            winnings = self.bet * 2
            self.message_label.config(text=f"Dealer busts! You win {winnings} euros.")
            self.balance += winnings
        elif player_value > dealer_value:
            winnings = self.bet * 2
            self.message_label.config(text=f"You win! You get {winnings} euros.")
            self.balance += winnings
        elif player_value == dealer_value:
            self.message_label.config(text=f"It's a tie! You get your bet of {self.bet} euros back.")
            self.balance += self.bet
        else:
            self.message_label.config(text=f"Dealer wins. You lose {self.bet} euros.")

        self.balance_label.config(text=f"Balance: {self.balance} euros")
        self.end_round()

    def end_round(self):
        self.button_frame.pack_forget()
        if self.balance > 0:
            self.bet_button.config(text="Place Bet", command=self.place_bet)
        else:
            self.message_label.config(text="Game over! You have run out of money.")
            self.bet_button.config(state=tk.DISABLED)

# Run the game
if __name__ == "__main__":
    root = tk.Tk()
    root.img = tk.PhotoImage(file='cards/red_joker.png')
    root.iconphoto(False, root.img)
    game = BlackjackGame(root)
    root.mainloop()